In [25]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import math

pd.set_option('display.max_columns', 50)

In [3]:
def data_import(csv):

  df = pd.read_csv(csv)
  df = df.drop(columns = ["OT_y.1", "OT_x.1", "Away_Win_y", "Pts_Away_y", "Home_Win_y", "Pts_Home_y", "OT_y", "Away_Win_x", "Away_Win_x", "Home_Win_x", "Pts_Home_x", "Pts_Away_x", "Pts_Home", "Pts_Away", "Away_Win"])

  return df
  

,Unnamed: 0,GAME_DATE_EST,GAME_ID,TEAM_CITY_NAME,TEAM_ABBREVIATION_x,TEAM_WINS_LOSSES_x,PTS_x,FG_PCT_x,FT_PCT_x,FG3_PCT_x,AST_x,REB_x,TOV_x,TEAM_ABBREVIATION_y,TEAM_WINS_LOSSES_y,PTS_y,FG_PCT_y,FT_PCT_y,FG3_PCT_y,AST_y,REB_y,TOV_y,OT_x,Games_in_2_x,Games_in_5_x,Games_in_7_x,Games_in_2_y,Games_in_5_y,Games_in_7_y,Home_Distance,Away_Distance,Dist_in_2_x,Dist_in_5_x,Dist_in_7_x,Dist_in_2_y,Dist_in_5_y,Dist_in_7_y,Home_Win
0,0,2018-10-16,21800001,Boston,BOS,1-0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,PHI,0-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,False,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,True
1,1,2018-10-16,21800002,Golden State,GSW,1-0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,OKC,0-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,False,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,True
2,2,2018-10-17,21800003,Charlotte,CHA,0-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,MIL,1-0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,False,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,False
3,3,2018-10-17,21800004,Detroit,DET,1-0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,BKN,0-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,False,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,True
4,4,2018-10-17,21800005,Indiana,IND,1-0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,MEM,0-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,False,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,1225,2019-04-10,21801226,Milwaukee,MIL,60-22,109.481481,0.477580,0.802852,0.357593,22.679012,41.987654,11.790123,OKC,49-33,110.049383,0.470519,0.772938,0.366728,25.185185,44.481481,14.358025,False,0,2,3,1,2,3,0.000000,1179.275404,0.000000,1116.887139,1250.581409,2296.868117,3414.460829,3414.460829,False
1226,1226,2019-04-10,21801227,San Antonio,SAS,48-34,111.592593,0.483012,0.775469,0.360630,26.728395,44.259259,14.358025,DAL,33-49,102.790123,0.458444,0.777309,0.352494,22.790123,44.296296,12.629630,False,0,1,2,1,2,3,2019.362731,404.939354,2019.362731,2509.915869,4910.098187,1080.811386,1756.683418,1756.683418,True
1227,1227,2019-04-10,21801228,Denver,DEN,54-28,107.493827,0.451988,0.714593,0.350753,21.209877,45.061728,13.432099,MIN,36-46,99.037037,0.443765,0.780432,0.350148,21.543210,40.518519,14.432099,False,1,2,3,1,2,3,595.877137,1124.440778,1616.305585,3193.548170,3193.548170,1124.440778,1124.440778,2513.931232,True
1228,1228,2019-04-10,21801229,LA,LAC,48-34,109.098765,0.472704,0.743148,0.351383,22.283951,43.925926,14.037037,UTA,50-32,107.987654,0.460691,0.716519,0.338790,23.753086,46.345679,15.296296,True,0,1,2,1,2,3,543.758097,935.332592,543.758097,1087.516194,1087.516194,1870.665185,2805.997777,3620.617762,True


In [35]:
def calc_win_perc(df):
  df = (df.assign(Home_Wins= df.TEAM_WINS_LOSSES_x.str.split('-').str.get(0).astype(int),
                            Home_Losses = df.TEAM_WINS_LOSSES_x.str.split('-').str.get(-1).astype(int)))

  df = (df.assign(Away_Wins= df.TEAM_WINS_LOSSES_y.str.split('-').str.get(0).astype(int),
                            Away_Losses = df.TEAM_WINS_LOSSES_y.str.split('-').str.get(-1).astype(int)))

  df["Home_Win%"] = df["Home_Wins"] / (df["Home_Wins"] + df["Home_Losses"])
  df["Away_Win%"] = df["Away_Wins"] / (df["Away_Wins"] + df["Away_Losses"])
  df["Win_Perc_Diff"] = df["Home_Win%"] - df["Away_Win%"]

  return df

,Unnamed: 0,GAME_DATE_EST,GAME_ID,TEAM_CITY_NAME,TEAM_ABBREVIATION_x,TEAM_WINS_LOSSES_x,PTS_x,FG_PCT_x,FT_PCT_x,FG3_PCT_x,AST_x,REB_x,TOV_x,TEAM_ABBREVIATION_y,TEAM_WINS_LOSSES_y,PTS_y,FG_PCT_y,FT_PCT_y,FG3_PCT_y,AST_y,REB_y,TOV_y,OT_x,Games_in_2_x,Games_in_5_x,Games_in_7_x,Games_in_2_y,Games_in_5_y,Games_in_7_y,Home_Distance,Away_Distance,Dist_in_2_x,Dist_in_5_x,Dist_in_7_x,Dist_in_2_y,Dist_in_5_y,Dist_in_7_y,Home_Win,Home_Wins,Home_Losses,Away_Wins,Away_Losses,Home_Win%,Away_Win%,Win_Perc_Diff
0,0,2018-10-16,21800001,Boston,BOS,1-0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,PHI,0-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,False,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,True,1,0,0,1,1.000000,0.000000,1.000000
1,1,2018-10-16,21800002,Golden State,GSW,1-0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,OKC,0-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,False,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,True,1,0,0,1,1.000000,0.000000,1.000000
2,2,2018-10-17,21800003,Charlotte,CHA,0-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,MIL,1-0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,False,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,False,0,1,1,0,0.000000,1.000000,-1.000000
3,3,2018-10-17,21800004,Detroit,DET,1-0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,BKN,0-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,False,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,True,1,0,0,1,1.000000,0.000000,1.000000
4,4,2018-10-17,21800005,Indiana,IND,1-0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,MEM,0-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,False,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,True,1,0,0,1,1.000000,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,1225,2019-04-10,21801226,Milwaukee,MIL,60-22,109.481481,0.477580,0.802852,0.357593,22.679012,41.987654,11.790123,OKC,49-33,110.049383,0.470519,0.772938,0.366728,25.185185,44.481481,14.358025,False,0,2,3,1,2,3,0.000000,1179.275404,0.000000,1116.887139,1250.581409,2296.868117,3414.460829,3414.460829,False,60,22,49,33,0.731707,0.597561,0.134146
1226,1226,2019-04-10,21801227,San Antonio,SAS,48-34,111.592593,0.483012,0.775469,0.360630,26.728395,44.259259,14.358025,DAL,33-49,102.790123,0.458444,0.777309,0.352494,22.790123,44.296296,12.629630,False,0,1,2,1,2,3,2019.362731,404.939354,2019.362731,2509.915869,4910.098187,1080.811386,1756.683418,1756.683418,True,48,34,33,49,0.585366,0.402439,0.182927
1227,1227,2019-04-10,21801228,Denver,DEN,54-28,107.493827,0.451988,0.714593,0.350753,21.209877,45.061728,13.432099,MIN,36-46,99.037037,0.443765,0.780432,0.350148,21.543210,40.518519,14.432099,False,1,2,3,1,2,3,595.877137,1124.440778,1616.305585,3193.548170,3193.548170,1124.440778,1124.440778,2513.931232,True,54,28,36,46,0.658537,0.439024,0.219512
1228,1228,2019-04-10,21801229,LA,LAC,48-34,109.098765,0.472704,0.743148,0.351383,22.283951,43.925926,14.037037,UTA,50-32,107.987654,0.460691,0.716519,0.338790,23.753086,46.345679,15.296296,True,0,1,2,1,2,3,543.758097,935.332592,543.758097,1087.516194,1087.516194,1870.665185,2805.997777,3620.617762,True,48,34,50,32,0.585366,0.609756,-0.024390


In [10]:
def create_baxter(df):

  df_bax = df.copy()

  df_bax = df_bax.drop(columns=['GAME_DATE_EST','TEAM_CITY_NAME','TEAM_ABBREVIATION_x','TEAM_WINS_LOSSES_x', 'TEAM_ABBREVIATION_y', 'TEAM_WINS_LOSSES_y'], axis = 1)

  df_bax["baxter_home"] = (df_bax['Dist_in_2_x'] > df_bax.Dist_in_2_x.quantile(0.975)) | (df_bax['Dist_in_5_x'] > df_bax.Dist_in_5_x.quantile(0.975)) | (df_bax['Dist_in_7_x'] > df_bax.Dist_in_7_x.quantile(0.975))
  df_bax["baxter_away"] = (df_bax['Dist_in_2_y'] > df_bax.Dist_in_2_y.quantile(0.975)) | (df_bax['Dist_in_5_y'] > df_bax.Dist_in_5_y.quantile(0.975)) | (df_bax['Dist_in_7_y'] > df_bax.Dist_in_7_y.quantile(0.975))

  return df_bax 


In [19]:
def create_train_test(df_bax):

  X = df_bax[['baxter_home', 'Win_Perc_Diff', 'FG_PCT_x', 'FG_PCT_y']]

  y = df_bax["Home_Win"]

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

  return X_train, X_test, y_train, y_test

In [16]:
def lr_model_train (X_train, y_train):

  lrmodel = LogisticRegression(random_state=1)

  lrmodel.fit(X_train, y_train)

  return lrmodel

def lr_model_preict(model, X_test):

  y_pred = model.predict(X_test)

  return model, y_pred

def lr_confusion_matrix(model, y_test, y_pred):
  
  cm = confusion_matrix(y_test, y_pred)
  return cm


def model_accuracy(y_test, y_pred):

  print(accuracy_score(y_test, y_pred))

[[ 70  62]
 [ 27 149]]
0.711038961038961


/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
